# NFL Verse Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nfl_data_py as nfl
import seaborn as sns

In [5]:
years = range(2010, 2024)

# Get the play-by-play data
pbp = nfl.import_pbp_data(years, downcast=True, cache=False, alt_path=None)

# Schedule
s = nfl.import_schedules(years)

2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [7]:
pbp.to_csv('../data/processed/NFL_verse_pbp_data.csv', index=False)
s.to_csv('../data/processed/NFL_verse_schedules.csv', index=False)